# Protein embeddings improve phage-host interaction prediction

**Mark Edward M. Gonzales<sup>1, 2</sup> & Anish M.S. Shrestha<sup>1, 2</sup>**

<sup>1</sup> Department of Software Technology, College of Computer Studies, De La Salle University, Manila, Philippines <br>
<sup>2</sup> Bioinformatics Laboratory, Advanced Research Institute for Informatics, Computing and Networking, De La Salle University, Manila, Philippines

{mark_gonzales, anish.shrestha}@dlsu.edu.ph

<hr>

## 📁 Output Files
If you would like to skip running this notebook, you may download the 

Intermediate output files (i.e., those saved in `temp/preprocessing`) should have already been included when the repository was cloned.

<hr>

# Part I: Preliminaries

Import the necessary libraries and modules.

In [1]:
import os
import shutil
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ConstantsUtil import ConstantsUtil
from SequenceParsingUtil import SequenceParsingUtil

%load_ext autoreload
%autoreload 2

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

pd.options.mode.chained_assignment = None

The parameter of `ConstantsUtil` corresponds to the download date of dataset.

In [3]:
constants = ConstantsUtil('16Sep2022')
util = SequenceParsingUtil(constants.DISPLAY_PROGRESS, constants.MISSPELLING_THRESHOLD, constants.MIN_LEN_KEYWORD)

To load this month's version of the dataset via INPHARED:

1. Clone the latest version of INPHARED from its official [repository](http://github.com/RyanCook94/inphared).
2. Download the PHROG HMMs (`all_phrogs.hmm`) from [Millard Lab](https://millardlab.org/2021/11/21/phage-annotation-with-phrogs/).
3. Run the following script to run the INPHARED pipeline (refer to the [documentation](https://github.com/RyanCook94/inphared#usage) for details): <br>
    `perl inphared.pl -e exclusion_list.txt -P all_phrogs.hmm/all_phrogs.hmm -c 0`
4. Move the following files to the `inphared` directory: <br>
    4.1. `<download_date>_data_excluding_refseq.tsv` <br>
    4.2. `<download_date>_phages_downloaded_from_genbank.gb` <br>
   Note: `<download_date>` should be the same as the argument passed to `ConstantsUtil` in the previous code block.

In [4]:
inphared_raw = pd.read_csv(f'{constants.INPHARED_TSV}', sep='\t')
inphared_raw = inphared_raw.drop_duplicates(ignore_index = True)
inphared_raw.head()

,Accession,Description,Classification,Genome Length (bp),Jumbophage,molGC (%),Molecule,Modification Date,Number CDS,Positive Strand (%),Negative Strand (%),Coding Capacity(%),Low Coding Capacity Warning,tRNAs,Host,Lowest Taxa,Genus,Sub-family,Family,Order,Class,Phylum,Kingdom,Realm,Baltimore Group,Genbank Division,Isolation Host (beware inconsistent and nonsense values)
0,LR756511,uncultured phage,uncultured phage environmental samples Viruses,595163,True,42.110,DNA,26-MAR-2020,1080,20.648148,79.351852,93.248236,NaN,62,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified
1,LR756508,uncultured phage,uncultured phage environmental samples Viruses,484177,True,39.553,DNA,26-MAR-2020,683,98.389458,1.610542,89.636022,NaN,31,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified
2,LR756504,uncultured phage,uncultured phage environmental samples Viruses,636363,True,26.393,DNA,26-MAR-2020,920,51.956522,48.043478,92.786193,NaN,34,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified
3,LR756503,uncultured phage,uncultured phage environmental samples Viruses,735411,True,32.225,DNA,26-MAR-2020,1014,30.276134,69.723866,93.113239,NaN,56,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified
4,LR756502,uncultured phage,uncultured phage environmental samples Viruses,642428,True,31.472,DNA,26-MAR-2020,971,54.582904,45.417096,94.525145,NaN,45,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified


In [5]:
inphared_raw.shape

(18389, 27)

<hr>

# Part II: Preprocessing of Host Information

Fo entries with `Unspecified` host in INPHARED, obtain the isolation host information from GenBank, if it makes sense.

In [6]:
util.set_inphared_gb(constants.INPHARED_GB)

In [7]:
inphared_unspec_host = inphared_raw.loc[inphared_raw['Host'] == 'Unspecified']
inphared_unspec_host.reset_index(inplace = True, drop = True)
inphared_unspec_host.head()

,Accession,Description,Classification,Genome Length (bp),Jumbophage,molGC (%),Molecule,Modification Date,Number CDS,Positive Strand (%),Negative Strand (%),Coding Capacity(%),Low Coding Capacity Warning,tRNAs,Host,Lowest Taxa,Genus,Sub-family,Family,Order,Class,Phylum,Kingdom,Realm,Baltimore Group,Genbank Division,Isolation Host (beware inconsistent and nonsense values)
0,LR756511,uncultured phage,uncultured phage environmental samples Viruses,595163,True,42.110,DNA,26-MAR-2020,1080,20.648148,79.351852,93.248236,NaN,62,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified
1,LR756508,uncultured phage,uncultured phage environmental samples Viruses,484177,True,39.553,DNA,26-MAR-2020,683,98.389458,1.610542,89.636022,NaN,31,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified
2,LR756504,uncultured phage,uncultured phage environmental samples Viruses,636363,True,26.393,DNA,26-MAR-2020,920,51.956522,48.043478,92.786193,NaN,34,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified
3,LR756503,uncultured phage,uncultured phage environmental samples Viruses,735411,True,32.225,DNA,26-MAR-2020,1014,30.276134,69.723866,93.113239,NaN,56,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified
4,LR756502,uncultured phage,uncultured phage environmental samples Viruses,642428,True,31.472,DNA,26-MAR-2020,971,54.582904,45.417096,94.525145,NaN,45,Unspecified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,Unclassified,ENV,Unspecified


In [8]:
phages_unspec_host = util.get_phages_unspec_host(inphared_unspec_host)

Processed 1000 records
Processed 2000 records
Processed 3000 records
Processed 4000 records
Processed 5000 records
Processed 6000 records
Processed 7000 records
Processed 8000 records
Processed 9000 records
Processed 10000 records
Processed 11000 records
Processed 12000 records
Processed 13000 records
Processed 14000 records
Processed 15000 records
Processed 16000 records
Processed 17000 records
Processed 18000 records
Processed 19000 records
Processed 20000 records
Processed 21000 records
Processed 22000 records
Processed 23000 records
Processed 24000 records
Processed 25000 records
Processed 26000 records
Processed 27000 records
Processed 28000 records
Processed 29000 records
Processed 30000 records
Processed 31000 records


In [9]:
unfiltered_hosts = util.get_unfiltered_hosts()
genus_typo = util.get_genus_typo(f'{constants.GENUS_TYPO}')
unfiltered_suspected_genera = util.get_unfiltered_suspected_genera(constants.CANDIDATE_REGEX)

excluded_hosts = util.get_excluded_hosts(constants.ARCHAEA, constants.BACTERIA_NOT_GENUS, constants.EXCLUDED_HOSTS)
valid_hosts = util.get_valid_hosts()

Add another layer of checking to identify typos in GenBank annotations, as well as oversights during manual filtering:
- Recheck genera in `valid_hosts` that are not in INPHARED's `Host` column.
- Identify entries with minimum edit distance &leq; 2 (likely misspellings).
<br>

Adjust the exclusion and typo correction files in the `preprocessing` directory accordingly, and rerun the previous cell, as needed.

In [10]:
inphared_hosts = set(inphared_raw['Host'].str.lower())
valid_hosts - inphared_hosts

{'silvanigrella'}

In [11]:
valid_hosts_list = list(valid_hosts)
for i in range(len(valid_hosts_list)):
    for j in range(i + 1, len(valid_hosts_list)):
        if util.is_possible_misspelling(valid_hosts_list[i], valid_hosts_list[j]):
            print(valid_hosts_list[i], valid_hosts_list[j])

microbacterium mycobacterium


Update `host` column based on the manually filtered info from GenBank.

In [12]:
inphared_augmented = inphared_raw.copy(deep = True)
util.update_host_column(constants.CANDIDATE_REGEX, inphared_unspec_host, inphared_augmented)

inphared_augmented['Host'] = inphared_augmented['Host'].str.lower()
inphared_augmented['Host'].value_counts()

Index: 424 | Name: ON970599 | Host: gordonia
Processed 1000 records
Processed 2000 records
Index: 2105 | Name: MN940411 | Host: staphylococcus
Processed 3000 records
Index: 3471 | Name: AF125163 | Host: vibrio
Index: 3477 | Name: AF063097 | Host: escherichia
Processed 4000 records
Index: 3576 | Name: OK499975 | Host: proteus
Index: 3705 | Name: OK649958 | Host: haloferax
Index: 3761 | Name: MW822148 | Host: mycobacterium
Processed 5000 records
Index: 4552 | Name: MH590600 | Host: microbacterium
Index: 4628 | Name: MH271321 | Host: microbacterium
Index: 4643 | Name: MH155873 | Host: microbacterium
Index: 5392 | Name: LC625742 | Host: enterococcus
Processed 6000 records
Processed 7000 records
Processed 8000 records
Processed 9000 records
Index: 5662 | Name: MT588083 | Host: salmonella
Processed 10000 records
Processed 11000 records
Processed 12000 records
Processed 13000 records
Processed 14000 records
Processed 15000 records
Processed 16000 records
Processed 17000 records
Processed 1800

unspecified                                     2566
mycobacterium                                   2189
escherichia                                     1627
vibrio                                           852
pseudomonas                                      826
salmonella                                       744
streptococcus                                    730
klebsiella                                       628
gordonia                                         579
staphylococcus                                   508
enterobacteria                                   498
microbacterium                                   459
bacillus                                         377
lactococcus                                      364
synechococcus                                    362
arthrobacter                                     340
streptomyces                                     310
flavobacterium                                   247
acinetobacter                                 

### Polyvalent Phages

There are only few polyvalent (multi-host) phages in our database. Hence, for simplicity, we map each polyvalent phage only to its host with the highest number of interacting phages in the dataset.

In [13]:
multiple_hosts = set()

for host, count in dict(inphared_augmented['Host'].value_counts()).items():
    if '|' in host:
        multiple_hosts.add(host)
        print(host, ':', count)

escherichia | chryseobacterium | pseudomonas : 4
phormidium | plectonema : 1


In [14]:
for host in multiple_hosts:
    # We have confirmed that the first host is the host with the highest number of interacting phages in the dataset.
    inphared_augmented.loc[inphared_augmented['Host'] == host, 'Host'] = host.split(' | ')[0]

In [15]:
inphared = inphared_augmented.loc[inphared_augmented['Host'] != 'unspecified']
inphared.reset_index(inplace = True, drop = True)
inphared.head()

,Accession,Description,Classification,Genome Length (bp),Jumbophage,molGC (%),Molecule,Modification Date,Number CDS,Positive Strand (%),Negative Strand (%),Coding Capacity(%),Low Coding Capacity Warning,tRNAs,Host,Lowest Taxa,Genus,Sub-family,Family,Order,Class,Phylum,Kingdom,Realm,Baltimore Group,Genbank Division,Isolation Host (beware inconsistent and nonsense values)
0,MK250029,Prevotella phage Lak-C1,Prevotella phage Lak-C1 Myoviridae Caudovirice...,540217,True,25.796,DNA,13-JAN-2019,830,47.108434,52.891566,68.324951,NaN,30,prevotella,Myoviridae,Unclassified,Unclassified,Myoviridae,Unclassified,Caudoviricetes,Uroviricota,Heunggongvirae,Duplodnaviria,Group I,ENV,Prevotella sp.
1,MK250028,Prevotella phage Lak-B9,Prevotella phage Lak-B9 Myoviridae Caudovirice...,550053,True,26.012,DNA,13-JAN-2019,859,52.270081,47.729919,69.188424,NaN,29,prevotella,Myoviridae,Unclassified,Unclassified,Myoviridae,Unclassified,Caudoviricetes,Uroviricota,Heunggongvirae,Duplodnaviria,Group I,ENV,Prevotella sp.
2,MK250027,Prevotella phage Lak-B8,Prevotella phage Lak-B8 Myoviridae Caudovirice...,551627,True,26.022,DNA,13-JAN-2019,860,53.023256,46.976744,69.318761,NaN,33,prevotella,Myoviridae,Unclassified,Unclassified,Myoviridae,Unclassified,Caudoviricetes,Uroviricota,Heunggongvirae,Duplodnaviria,Group I,ENV,Prevotella sp.
3,MK250026,Prevotella phage Lak-B7,Prevotella phage Lak-B7 Myoviridae Caudovirice...,550702,True,26.020,DNA,13-JAN-2019,859,53.201397,46.798603,69.363285,NaN,33,prevotella,Myoviridae,Unclassified,Unclassified,Myoviridae,Unclassified,Caudoviricetes,Uroviricota,Heunggongvirae,Duplodnaviria,Group I,ENV,Prevotella sp.
4,MK250025,Prevotella phage Lak-B6,Prevotella phage Lak-B6 Myoviridae Caudovirice...,546689,True,26.029,DNA,13-JAN-2019,847,52.656434,47.343566,69.118274,NaN,30,prevotella,Myoviridae,Unclassified,Unclassified,Myoviridae,Unclassified,Caudoviricetes,Uroviricota,Heunggongvirae,Duplodnaviria,Group I,ENV,Prevotella sp.


Replace the genera of some hosts with standard NCBI Taxonomy nomenclature.

In [16]:
inphared['Host'].replace(util.get_ncbi_standard_nomenclature(constants.NCBI_STANDARD_NOMENCLATURE), inplace = True)
inphared['Host'].value_counts()

mycobacterium                2189
escherichia                  1632
vibrio                        852
pseudomonas                   826
salmonella                    744
streptococcus                 730
klebsiella                    628
gordonia                      579
enterobacter                  558
staphylococcus                508
microbacterium                459
bacillus                      377
lactococcus                   364
synechococcus                 362
arthrobacter                  340
streptomyces                  310
flavobacterium                247
acinetobacter                 196
enterococcus                  185
rhizobium                     154
aeromonas                     140
erwinia                       137
shigella                      128
propionibacterium             125
yersinia                      120
rheinheimera                  110
campylobacter                  95
pectobacterium                 92
xanthomonas                    91
lactobacillus 

Just verifying that all the entries are unique

In [17]:
inphared.shape[0] == inphared['Accession'].nunique()

True

###  Some Statistics

Statistics on phages

In [18]:
print("Original INPHARED:\t\t", inphared_raw.shape[0])
print("With Specified Hosts:\t\t", inphared_raw.shape[0] - inphared_unspec_host.shape[0])
print("Plus Manually Filtered Hosts:\t", inphared.shape[0])

Original INPHARED:		 18389
With Specified Hosts:		 15739
Plus Manually Filtered Hosts:	 15823


Statistics on host genera

In [19]:
print("Num. of Unique Host Genera:\t", inphared['Host'].nunique())

Num. of Unique Host Genera:	 279


Save the entries with host information to a CSV file.

In [20]:
inphared.to_csv(os.path.join(constants.TEMP_PREPROCESSING, constants.INPHARED_WITH_HOSTS), index = False)

<hr>

# Part III: Selection of Annotated Receptor-Binding Proteins (RBPs)

Load only the phage entries with host data.

In [21]:
inphared = pd.read_csv(f'{constants.TEMP_PREPROCESSING}/{constants.INPHARED_WITH_HOSTS}')
util.set_inphared(inphared)

inphared.shape

(15823, 27)

Identify which phage entries have CDS information and which do not.

In [22]:
no_cds_annot = util.get_no_cds_annot()

Processed 1000 records
Processed 2000 records
Processed 3000 records
Processed 4000 records
Processed 5000 records
Processed 6000 records
Processed 7000 records
Processed 8000 records
Processed 9000 records
Processed 10000 records
Processed 11000 records
Processed 12000 records
Processed 13000 records
Processed 14000 records
Processed 15000 records
Processed 16000 records
Processed 17000 records
Processed 18000 records
Processed 19000 records
Processed 20000 records
Processed 21000 records
Processed 22000 records
Processed 23000 records
Processed 24000 records
Processed 25000 records
Processed 26000 records
Processed 27000 records
Processed 28000 records
Processed 29000 records
Processed 30000 records
Processed 31000 records


In [23]:
with open(os.path.join(constants.TEMP_PREPROCESSING, constants.NO_CDS_ANNOT),'wb') as no_cds_annot_file:
    pickle.dump(no_cds_annot, no_cds_annot_file)

In [24]:
with open(f'{constants.TEMP_PREPROCESSING}/{constants.NO_CDS_ANNOT}','rb') as no_cds_annot_file:
    no_cds_annot = pickle.load(no_cds_annot_file)
    
util.set_no_cds_annot(no_cds_annot)

### Some statistics

In [25]:
print("Without CDS info:\t", len(no_cds_annot))
print("With CDS info:\t\t", inphared.shape[0] - len(no_cds_annot))

Without CDS info:	 665
With CDS info:		 15158


## *A. Process entries with CDS information*

- Use regex from this [RBP prediction study](https://www.mdpi.com/1999-4915/14/6/1329) by Boeckaerts <i>et al.</i> (2022) to select the annotated RBPs
   - We modify this regex to:
     - Make more accommodating to typos (e.g., look for `bind` instead of `binding`)
     - Allow multiple spaces between tokens
   - For comparison:
     - Original regex: `tail.?(?:spike|fib(?:er|re))|^recept(?:o|e)r.?(?:binding|recognizing).*(?:protein)?|^RBP`
     - Modified regex: `tail?(.?|\s*)(?:spike?|fib(?:er|re))|recept(?:o|e)r(.?|\s*)(?:bind|recogn).*(?:protein)?|(?<!\w)RBP(?!a)`
     - Annotations captured by modified regex but not by old regex:
       - `receptor recognition protein`
   - The same [study](https://www.mdpi.com/1999-4915/14/6/1329) has an exclusion list that covers proteins related to RBPs but are not RBPs themselves (e.g., assembly and portal proteins).
       <br><br>

- If product does not match regex for RBP, check if it is a hypothetical protein
   - The same [study](https://www.mdpi.com/1999-4915/14/6/1329) has a list of keywords associated with hypothetical proteins
      - We remove `putative`, `probable`, and `probably` since they are indicative of a product with a putative (rather than an unknown) function.
   - Make more accommodating to typos by allowing keywords with minimum edit distance ≤ 2 (likely misspellings)
   - Create a manual list to handle cases where the keyword `hypothetical` is present but the putative function is already given, as in the case of:
       - `conserved hypothetical lipoprotein`
       - `hypothetical host-like ribonucleoside diphosphate reductase`       <br><br>
      
- Predict whether these hypothetical proteins are RBPs using the XGBoost model from the [RBP prediction study](https://www.mdpi.com/1999-4915/14/6/1329) by Boeckaerts <i>et al.</i> (2022).

In [26]:
util.set_token_delimiter(constants.TOKEN_DELIMITER)
util.construct_keyword_list(constants.HYPOTHETICAL_KEYWORDS, constants.RBP_RELATED_NOT_RBP, constants.PUTATIVE_FUNCTIONS)

In [ ]:
annot_products = util.get_annot_products()

Processed 1000 records
Processed 2000 records
Processed 3000 records
Processed 4000 records
Processed 5000 records
Processed 6000 records
Processed 7000 records
Processed 8000 records
Processed 9000 records
Processed 10000 records
Processed 11000 records
Processed 12000 records
Processed 13000 records
Processed 14000 records
Processed 15000 records
Processed 16000 records
Processed 17000 records
Processed 18000 records
Processed 19000 records
Processed 20000 records
Processed 21000 records
Processed 22000 records
Processed 23000 records


In [ ]:
with open(os.path.join(constants.TEMP_PREPROCESSING, constants.ANNOT_PRODUCTS),'wb') as annot_products_file:
    pickle.dump(annot_products, annot_products_file)

In [ ]:
with open(f'{constants.TEMP_PREPROCESSING}/{constants.ANNOT_PRODUCTS}','rb') as annot_products_file:
    annot_products = pickle.load(annot_products_file)
    
util.set_annot_products(annot_products)
for annot_product in sorted(list(annot_products)):
    print(annot_product)

In [ ]:
rbp_products, hypothetical_proteins = util.get_rbp_hypothetical_proteins(constants.RBP_REGEX)

In [ ]:
with open(os.path.join(constants.TEMP_PREPROCESSING, constants.RBP_PRODUCTS),'wb') as rbp_products_file:
    pickle.dump(rbp_products, rbp_products_file)
    
with open(os.path.join(constants.TEMP_PREPROCESSING, constants.HYPOTHETICAL_PRODUCTS),'wb') as hypothetical_proteins_file:
    pickle.dump(hypothetical_proteins, hypothetical_proteins_file)

In [ ]:
with open(f'{constants.TEMP_PREPROCESSING}/{constants.RBP_PRODUCTS}','rb') as rbp_products_file:
    rbp_products = pickle.load(rbp_products_file)
    
util.set_rbp_products(rbp_products)
rbp_products

In [ ]:
with open(f'{constants.TEMP_PREPROCESSING}/{constants.HYPOTHETICAL_PRODUCTS}','rb') as hypothetical_proteins_file:
    hypothetical_proteins = pickle.load(hypothetical_proteins_file)
    
util.set_hypothetical_proteins(hypothetical_proteins)
for protein in hypothetical_proteins:
    print(protein)

### Get distribution of RBP lengths

This preliminary analysis of the distribution of the RBP lengths covers both the sequences with CDS information and those where the CDS information features are predicted via Prokka. We carry it now in order since we need the lower and upper bounds for RBP lengths to generate the FASTA files.

In [ ]:
len_distribution = util.generate_rbp_len_distribution()
util.generate_rbp_len_distribution_prokka(len_distribution, constants.INPHARED_GENOME)

lengths = []
for length, freq in len_distribution.items():
    for _ in range(freq):
        lengths.append(length)

In [ ]:
with open(os.path.join(constants.TEMP_PREPROCESSING, constants.RBP_LENGTHS),'wb') as rbp_lengths_file:
    pickle.dump(lengths, rbp_lengths_file)

Statistically compute the lower and upper bounds for RBP lengths.

In [ ]:
with open(f'{constants.TEMP_PREPROCESSING}/{constants.RBP_LENGTHS}','rb') as rbp_lengths_file:
    lengths = pickle.load(rbp_lengths_file)

Q1 = np.percentile(lengths, 25, interpolation = 'midpoint')
Q3 = np.percentile(lengths, 75, interpolation = 'midpoint')
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print("Lower bound:", lower_bound)
print("Upper bound:", upper_bound)

### Generate FASTA for entries with CDS information

Generate FASTA files containing protein sequences of hypothetical proteins and RBPs

In [ ]:
RBP_NEW_DIR = f'{constants.INPHARED}/{constants.FASTA}/{constants.RBP}/{constants.GENBANK}'
HYPOTHETICAL_NEW_DIR = f'{constants.INPHARED}/{constants.FASTA}/{constants.HYPOTHETICAL}/{constants.GENBANK}'

if not os.path.exists(RBP_NEW_DIR):
    os.mkdir(RBP_NEW_DIR)
    
if not os.path.exists(HYPOTHETICAL_NEW_DIR):
    os.mkdir(HYPOTHETICAL_NEW_DIR)

In [ ]:
util.generate_rbp_hypothetical_fasta(RBP_NEW_DIR, HYPOTHETICAL_NEW_DIR, constants.LOWER_BOUND_RBP_LENGTH, 
                                     constants.UPPER_BOUND_RBP_LENGTH)

The FASTA files are stored in the `inphared/fasta` directory, specifically in the `genbank` subdirectories.

## *B. Process entries without CDS information*

- Annotate genes using [Prokka](https://academic.oup.com/bioinformatics/article/30/14/2068/2390517), which calls [Prodigal](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-11-119) to predict coordinates of ORFs.
- Configure Prokka to perform functional annotation using [PHROGs](https://academic.oup.com/nargab/article/3/3/lqab067/6342220).
- Process in the same manner as the entries with CDS information.

In [ ]:
RBP_NEW_DIR = f'{constants.INPHARED}/{constants.FASTA}/{constants.RBP}/{constants.PROKKA}'
HYPOTHETICAL_NEW_DIR = f'{constants.INPHARED}/{constants.FASTA}/{constants.HYPOTHETICAL}/{constants.PROKKA}'

if not os.path.exists(RBP_NEW_DIR):
    os.mkdir(RBP_NEW_DIR)
    
if not os.path.exists(HYPOTHETICAL_NEW_DIR):
    os.mkdir(HYPOTHETICAL_NEW_DIR)

Analyze the gene product annotations returned by PHROG.

In [ ]:
annot_products_prokka = util.get_annot_products_prokka(constants.INPHARED_GENOME)

In [ ]:
x, y = util.get_rbp_hypothetical_proteins_prokka(constants.RBP_REGEX, annot_products_prokka)

Combine the keywords for RBPs and hypothetical proteins from GenBank (i.e., entries with CDS information) and PHROG annotations (i.e., entries without CDS information).

In [ ]:
util.set_rbp_products(rbp_products.union(x))
util.set_hypothetical_proteins(hypothetical_proteins.union(y))

In [ ]:
rbp_products = rbp_products.union(x)
hypothetical_proteins = hypothetical_proteins.union(y)

In [ ]:
with open(os.path.join(constants.TEMP_PREPROCESSING, constants.RBP_PRODUCTS),'wb') as rbp_products_file:
    pickle.dump(rbp_products, rbp_products_file)
    
with open(os.path.join(constants.TEMP_PREPROCESSING, constants.HYPOTHETICAL_PRODUCTS),'wb') as hypothetical_proteins_file:
    pickle.dump(hypothetical_proteins, hypothetical_proteins_file)

### Generate FASTA for entries without CDS information

Generate FASTA files containing protein sequences of hypothetical proteins and RBPs

In [ ]:
util.generate_rbp_hypothetical_fasta_prokka(constants.INPHARED_GENOME, RBP_NEW_DIR, HYPOTHETICAL_NEW_DIR, 
                                            lower_bound, upper_bound)

The FASTA files are stored in the `inphared/fasta` directory, specifically in the `prokka` subdirectories.

### C. *Generate FFN containing the genomes of RBPs and hypothetical proteins*

In [ ]:
util.generate_rbp_hypothetical_nucleotide(f'{constants.INPHARED}/{constants.FASTA}/{constants.NUCLEOTIDE}/{constants.RBP}/{constants.GENBANK}',
                                          f'{constants.INPHARED}/{constants.FASTA}/{constants.NUCLEOTIDE}/{constants.HYPOTHETICAL}/{constants.GENBANK}', 
                                          lower_bound, upper_bound)

The FFN files are stored in the `inphared/fasta/nucleotide` directory, specifically in the `genbank` subdirectories.

In [ ]:
util.generate_rbp_hypothetical_nucleotide_prokka(constants.INPHARED_GENOME,
                                                 f'{constants.INPHARED}/{constants.FASTA}/{constants.NUCLEOTIDE}/{constants.RBP}/{constants.PROKKA}',
                                                 f'{constants.INPHARED}/{constants.FASTA}/{constants.NUCLEOTIDE}/{constants.HYPOTHETICAL}/{constants.PROKKA}', 
                                                 lower_bound, upper_bound)

The FFN files are stored in the `inphared/fasta/nucleotide` directory, specifically in the `prokka` subdirectories.